### Normalización de datos fotómetricos

El objetivo de este algoritmo es normalizar los datos fotométricos. Para ello, en primer lugar, busca las coincidencias entre los identificadores de los espectros ya normalizados y los datos fotométricos con los mismos identificadores.

A continuación, se lleva a cabo la normalización utilizando la siguiente fórmula:

$$
X_{j}^{\text{norm}} = \frac{X_j - X_{j}^{\text{min}}}{X_{j}^{\text{max}} - X_{j}^{\text{min}}}
$$


In [ ]:
from astropy.io import fits
import numpy as np
import os

#### 1. Cargar, filtrar y extraer datos


In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
#Datos
ph_data = fits.open('/drive/My Drive/photo_data.fits') #open file
ident = fits.open('/drive/My Drive/Data/identifier/identifiers.fits')


Realizar seleccion cruzada para filtrar aquellos identificadores (fiberid, mjd,plate) de los datos fotométricos que se correspondan con los de los espectros normalizados (en el algoritmo normalized_spec)

In [ ]:
#extraer tablas
photo_table = ph_data[1].data
ident_table = ident[1].data

#extraer columnas necesarias
#de photo_data
plate_photo = photo_table['PLATE']
mjd_photo = photo_table['MJD']
fiberid_photo = photo_table['FIBERID']

#de identifiers
plate_id = ident_table['plate']
mjd_id = ident_table['mjd']
fiberid_id = ident_table['fiberid']

In [ ]:
#realizar una máscara
mask = np.zeros(len(photo_table), dtype=bool)

#recorrer los identifiers para marcar coincidencias
for i in range(len(ident_table)):
    match = (plate_photo == plate_id[i]) & (mjd_photo == mjd_id[i]) & (fiberid_photo == fiberid_id[i])
    mask |= match  #acumular las coincidencias

In [ ]:
#aplicar la máscara para quedarse solo con las coincidencias
filtered_data = photo_table[mask]
print(f"Se encontraron {len(filtered_data)} coincidencias.")

Se encontraron 110 coincidencias.


Guarda archivo con las coincidencias de la selección cruzada


In [ ]:
#crear una archivo fits con los datos filtrados

#definir la nueva carpeta donde guardar el archivo
photo_data= '/drive/My Drive/Data/photo_data'

#crear la carpeta si no existe
os.makedirs(photo_data, exist_ok=True)

#ruta final del archivo
ruta_archivo = os.path.join(photo_data, 'photo_data_filtrado.fits')

#crear y guardar el archivo fits
hdu = fits.BinTableHDU(data=filtered_data)
hdu.writeto(ruta_archivo, overwrite=True)
print(f"Archivo guardado como: {ruta_archivo}")

Archivo guardado como: /drive/My Drive/Data/photo_data/photo_data_filtrado.fits


Luego del filtrado con la selección cruzada, se realiza una extracción de las columnas necesarias (vector 15D) para normalizar los datos fotómetricos.


In [ ]:
pho_data = fits.open('/drive/My Drive/Data/photo_data/photo_data_filtrado.fits') #ruta donde estan los archivos fotométricos filtrados

#extraer la tabla
data = pho_data[1].data

#extraer las columnas con las que se busca trabajar (vector 15D con datos fotométricos)
selected_columns = ['MAG_U', 'MAG_G', 'MAG_R', 'MAG_I', 'MAG_Z',
                    'MAG_U-MAG_G', 'MAG_G-MAG_R', 'MAG_R-MAG_I', 'MAG_I-MAG_Z', 'MAG_U-MAG_Z',
                    'MODELUERR','MODELGERR','MODELRERR','MODELIERR', 'MODELZERR']


#convertir los datos a un array de numpy
#X = np.array([data[field] for field in selected_columns]).T  #.T es para transponer porque sino obtengo un arreglo que tiene (15, datos),
#es decir cada fila representa una de las 15 features y cada columna representa un dato de la muestra. y se necesita al reves (filas:datos, columnas: 15 features)

#extraer datos que se van a normalizar
X = np.array([data[col] for col in selected_columns]).T


In [ ]:
X

array([[1.94178867e+01, 1.79564133e+01, 1.71566181e+01, ...,
        6.89731911e-03, 5.90569992e-03, 1.38692185e-02],
       [1.93364601e+01, 1.81576443e+01, 1.74728737e+01, ...,
        7.18058553e-03, 7.49328174e-03, 2.02310458e-02],
       [1.93052311e+01, 1.74404144e+01, 1.65964355e+01, ...,
        4.02037567e-03, 3.75549961e-03, 6.62779761e-03],
       ...,
       [1.71055851e+01, 1.64006615e+01, 1.61457405e+01, ...,
        4.04061843e-03, 3.83140869e-03, 8.65597744e-03],
       [1.70133095e+01, 1.62770367e+01, 1.60988693e+01, ...,
        3.83778824e-03, 4.18931432e-03, 9.97402333e-03],
       [1.66233692e+01, 1.56920462e+01, 1.54076262e+01, ...,
        3.15016508e-03, 3.60669428e-03, 9.56163462e-03]], dtype=float32)

#### 2. Normalización de los datos


a. Se calculan los valores mínimos y máximos para cada una de los 15 *features*

b. Se aplica la transformación:

$$
X_{j}^{\text{norm}} = \frac{X_j - X_{j}^{\text{min}}}{X_{j}^{\text{max}} - X_{j}^{\text{min}}}
$$



In [ ]:
#calcular valores mínimos y máximos para cada photo feature
X_min = np.min(X, axis=0) #axis = 0 significa que la operación de sacar el mínimo o máximo se realiza a lo largo de las filas, para cada columna
X_max = np.max(X, axis=0)

In [ ]:
X_min

array([ 1.6623369e+01,  1.5692046e+01,  1.5407626e+01,  1.5220804e+01,
        1.5107402e+01,  7.0492363e-01,  1.3627815e-01,  1.0599518e-01,
       -7.0480347e-02,  1.1233377e+00,  1.2448050e-02,  3.1575679e-03,
        3.1501651e-03,  3.6066943e-03,  6.6277976e-03], dtype=float32)

In [ ]:
#normalizar los datos

#definir el denominador para que en caso de que sea cero, detenga el programa
denominator = X_max - X_min
if np.any(denominator == 0):
    print('Error: Hay al menos una característica con X_max == X_min, lo que haría que la normalización no sea válida')
    exit()  #detiene la ejecución del programa

#normalización de los datos
X_norm = (X - X_min) / denominator

In [ ]:
cols = pho_data[1].columns

#crear nuevas columnas para reemplazar las viejas en la tabla completa con los datos normalizados
new_columns = []
for col in cols:
    if col.name in selected_columns:
        i = selected_columns.index(col.name)
        new_col = fits.Column(name=col.name, format=col.format, array=X_norm[:, i])
        new_columns.append(new_col)
    else:
        new_columns.append(col)



#### 3. Crear el archivo con los datos fotométricos normalizados


In [ ]:
#crear la tabla con los datos normalizados
new_hdu = fits.BinTableHDU.from_columns(new_columns)

In [ ]:
#guardar la tabla
output_dir = '/drive/My Drive/Data/photo_data/'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'photo_data_normalizado.fits')

hdu.writeto(output_path, overwrite=True)

print(f'Datos normalizados guardados en {output_dir}')

Datos normalizados guardados en /drive/My Drive/Data/photo_data/
